# Data Rationalisation

This notebook prepares and inserts data from a `.csv` source into an existing database table.

## Imports

In [1]:
import pandas as pd
from redshift_connector import connect
import re

## Setup

In [2]:
def get_db_connection():
    return connect(host="c17-redshift-cluster.cdq12ms5gjyk.eu-west-2.redshift.amazonaws.com",
                   user="admin",
                   password="Password1",
                   database="dw_air_travel",
                   port=5439)

conn = get_db_connection()

In [3]:
def get_state_mapping(conn) -> dict:
    """Returns a mapping of state codes and DB IDs."""

    with conn.cursor() as cur:
        cur.execute("SELECT state_code, state_id FROM s_coach_dan.state;")
        data = cur.fetchall()

    mapping = {}
    for row in data:
        mapping[row[0]] = row[1]

    return mapping

def get_shape_mapping(conn) -> dict:
    """Returns a mapping of shape names and DB IDs."""

    with conn.cursor() as cur:
        cur.execute("SELECT ufo_shape_name, ufo_shape_id FROM ufo.ufo_shape;")
        data = cur.fetchall()

    mapping = {}
    for row in data:
        mapping[row[0]] = row[1]

    return mapping

## Data Sourcing

In [4]:
sightings = pd.read_csv("ufo_data.csv", sep="\t")

In [5]:
sightings.head(2)

,Unnamed: 0.1,Unnamed: 0,summary,city,state,date_time,shape,duration,stats,report_link,text,posted,city_latitude,city_longitude
0,0,0,My wife was driving southeast on a fairly popu...,Chester,VA,2019-12-12T18:43:00,light,"5 seconds, 0 minutes, 0 hours",Occurred : 12/12/2019 18:43 (Entered as : 12/...,http://www.nuforc.org/webreports/151/S151739.html,My wife was driving southeast on a fairly popu...,2019-12-22T00:00:00,37.343152,-77.408582
1,1,1,I think that I may caught a UFO on the NBC Nig...,Rocky Hill,CT,2019-03-22T18:30:00,circle,"5 seconds, 0 minutes, 0 hours",Occurred : 3/22/2019 18:30 (Entered as : 03/2...,http://www.nuforc.org/webreports/145/S145297.html,I think that I may caught a UFO on the NBC Nig...,2019-03-29T00:00:00,41.664800,-72.639300


## Data Cleaning

In [6]:
sightings = sightings.drop(columns=["Unnamed: 0.1", "Unnamed: 0", "city", "city_latitude", "city_longitude", "report_link",
                                    "posted", "summary", "stats"])

In [7]:
sightings["date_time"] = pd.to_datetime(sightings["date_time"])

In [8]:
state_mapping = get_state_mapping(conn)
shape_mapping = get_shape_mapping(conn)

In [9]:
sightings = sightings.dropna()

In [10]:
sightings["shape"] = sightings["shape"].map(shape_mapping)
sightings["state"] = sightings["state"].map(state_mapping)

sightings = sightings.dropna()
sightings["state"] = sightings["state"].astype(int)

In [11]:
def get_seconds_from_duration(text: str) -> int:
    """Returns the number of seconds from a complex time string."""

    nums = [int(n) for n in re.findall(r"(\d+)", text)]

    return nums[0] + nums[1] * 60 + nums[2] * 60

In [12]:
sightings["duration"] = sightings["duration"].apply(get_seconds_from_duration)

In [13]:
sightings.sample()

,state,date_time,shape,duration,text
20435,40,2010-02-28 23:00:00,1,180,Red blue green and white light slowly floating...


In [14]:
sightings.dtypes

state                 int64
date_time    datetime64[ns]
shape                 int64
duration              int64
text                 object
dtype: object

## Data Upload

In [15]:
sightings.to_csv("ufo_clean.csv", index=False, header=None)

In [16]:
with conn.cursor() as cur:
    cur.execute("set search_path to ufo;")
    cur.insert_data_bulk(filename="ufo_clean.csv", table_name="ufo.ufo_sighting_filled",
                         column_names=["state_id", "event_at", "shape_id", "duration_s", "event_description"],
                         parameter_indices=[0, 1, 2, 3, 4],
                         delimiter=",")

KeyboardInterrupt: 

## Clean up

In [ ]:
conn.commit()

In [ ]:
conn.close()